In [1]:
import tqdm
import pandas as pd
import string
import random
import numpy as np
import collections
import sklearn.model_selection
import sklearn.feature_extraction
import sklearn.feature_selection
import pomegranate
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('ticks')

import MeCab
# m = MeCab.Tagger(' -d /usr/lib/mecab/dic/mecab-ipadic-neologd -F%f[6],%f[0], -U%m,, -EEOS,EOS') # with POS
m = MeCab.Tagger(' -d /usr/lib/mecab/dic/mecab-ipadic-neologd -F%m,%f[0], -U%m,, -EEOS,EOS') # with POS
# m = MeCab.Tagger(' -d /usr/lib/mecab/dic/mecab-ipadic-neologd -F%f[6], -U%m,, -EEOS') # without POS

PATTERN_URL = """((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?≪≫“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""

In [2]:
df = pd.read_csv('talks.csv', header=None).dropna()
print(len(df))
df[:5]

689028


,0,1
0,・,やーだーイクの早いよー
1,・【画像あり】白人女,日本は暑いわね・・・(お尻バイン！)(太ももドン！)(ブラジャー透けッ・・・！)
2,・女子小学生,1時間くすぐりに耐えるだけで5万円くれるの？！
3,・彡(゜)(゜),あかん、ザクをぶった切ったらコロニーに穴開いてしもた
4,・フェネック,あんっあっアライさっぁっ


In [3]:
df_grouped = df.groupby(0).count().sort_values(by=1, ascending=False)
df_grouped[:5]

,1
0,
男,41771
女,15936
勇者,15482
俺,10762
妹,10246


In [4]:
popular_roles = df_grouped[:5].index
talks_by_roles = {role: df[df[0] == role][1].values for role in popular_roles}
talks_by_roles

{'俺': array(['おま〇こー？', 'チノちゃんのおま〇こっ！', 'おま〇こ♪おま〇こ♪', ..., 'ふう……ここまで逃げたらもう来ねえだろ',
        'た、助かった！おーい、おーい！', 'あじゅおじゅごぼうべおぼぶべばばっ'], dtype=object),
 '勇者': array(['おい、魔法使い', 'あ、失礼……まだ来客中だったか。しかし、変わった格好をしているが……', 'はじめまして、勇者です', ...,
        'はい。魔法使い殿の仰るとおりであります', 'ボクらも手紙書くよー！', 'はい！\u3000新人勇者、全力で頑張ります！'], dtype=object),
 '女': array(['！', 'いやぁぁぁっ！', '！', ..., '世間って狭いわよね', 'いいわよ。私はソルティドッグでも飲もうかしら',
        '私の名前は、志緒っていうの'], dtype=object),
 '妹': array(['それセ〇クスにならないの？', 'うーん', 'うん！それならいいよ', ..., '自己中死ね',
        'う、うぅ、てか私たちなんであんなに争ってたんだろ', 'てめぇは黙ってろ'], dtype=object),
 '男': array(['年金払ってるんですか', 'もらえるとき来るんですかね…', '見た目がおばあちゃんならもらえるかもわかんないですね', ...,
        'どっちだよ', '……分かった、出すぞ', 'ん……'], dtype=object)}

In [5]:
words_by_documents = [np.array(m.parse(talk).split(',')[::2]) for talk in talks_by_roles['勇者']]
poses_by_documents = [np.array(m.parse(talk).split(',')[1::2]) for talk in talks_by_roles['勇者']]
print(words_by_documents[0][:10])
print(poses_by_documents[0][:10])
set(poses_by_documents[0])

['おい' '、' '魔法使い' 'EOS']
['感動詞' '記号' '名詞' 'EOS']


{'EOS', '名詞', '感動詞', '記号'}

In [6]:
mm = pomegranate.MarkovChain.from_samples(words_by_documents[:500])

In [7]:
print(''.join(mm.sample(10)))

これから宿を包んだなEOSちょっと痛いん


In [ ]:
indices = [(poses == '名詞') + (poses == '形容詞') + (poses == '動詞') + (poses == '助詞') for poses in poses_by_documents]
words_by_documents = [words[index] for index, words in zip(indices, words_by_documents)]
poses_by_documents = [poses[index] for index, poses in zip(indices, poses_by_documents)]